In [1]:
import sys
import catboost as cb
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn as skl
from sklearn.metrics import confusion_matrix, accuracy_score, log_loss
import skopt
import scipy
from skopt.space import Real

PREDICCION_REAL = True
MAXIMIZAR_HIPERPARAMETROS = False
PARAMETROS = {
    'task_type' : 'GPU',
    'devices' : '0:1',
    'bootstrap_type' : 'MVS',
    'verbose' : True,
    'has_time' : True,
    
    #"rsm" : 0.36719138525672734,
    'bagging_temperature': 1.1063407351624084,
    'border_count': 7137,
    'depth': 9,
    'early_stopping_rounds': 4,
     'iterations': 7,
    'l2_leaf_reg': 0.17260178161990627,
    'learning_rate': 0.027307511040591478,
    'random_seed': 5712,
    'random_strength': 2.3464965049108726e-07,
    'scale_pos_weight': 0.819600044656442,
    'subsample': 0.9880746674599722
}

In [2]:
#APERTURA DE ARCHIVO DE ARCHIVOS
entrenamiento = pd.read_pickle("../Archivos/Arboles_entrenamiento.pkl")
test = pd.read_pickle("../Archivos/Arboles_validacion.pkl")

if (PREDICCION_REAL):
    entrenamiento = entrenamiento.append(test)
    test = pd.read_pickle("../Archivos/Arboles_test.pkl")

In [3]:
#entrenamiento['Sales_Contract_No'] = entrenamiento['Sales_Contract_No'].astype('category')
#entrenamiento = entrenamiento.sort_values('Opportunity_Created_Date')

In [4]:
#FILTRADO DE COLUMNAS - NO REMOVER STAGE O FECHA

#columnas_fecha = ['Month','Last_Modified_Date','Account_Created_Date','Opportunity_Created_Date','Quote_Expiry_Date','Planned_Delivery_Start_Date','Planned_Delivery_End_Date']
fugas = ['Sales_Contract_No','ID','Account_Name','Account_Owner','Opportunity_Owner','Last_Modified_By','ASP','ASP_(converted)']
otros = ['Currency']

entrenamiento = entrenamiento.drop(columns=fugas)
test = test.drop(columns=fugas)

#entrenamiento = entrenamiento.drop(columns=columnas_fecha)
#test = test.drop(columns=columnas_fecha)

In [5]:
#FECHAS A DIAS

columnas_fecha = ['Year-Month','Last_Modified_Date','Account_Created_Date','Opportunity_Created_Date','Quote_Expiry_Date','Planned_Delivery_Start_Date','Planned_Delivery_End_Date']
def fecha_a_dias(x):
    fecha_origen = pd.to_datetime('01/01/2000', format='%m/%d/%Y')
    for columna in columnas_fecha:
        x[columna] = x[columna].apply(lambda x : (x - fecha_origen).days)

fecha_a_dias(entrenamiento)
fecha_a_dias(test)

In [6]:
objetivo = entrenamiento['Stage']
entrenamiento = entrenamiento.drop(columns=['Stage'])
columnas_category = list(entrenamiento.select_dtypes(include=['category']).columns)
if 'Stage' in columnas_category : columnas_category.remove('Stage')

In [7]:
if not PREDICCION_REAL:
    test_label = test['Stage']
    test = test.drop(columns=['Stage'])

In [8]:
entrenamiento_pool = cb.Pool(entrenamiento, objetivo,cat_features=columnas_category)
test_pool = cb.Pool(test, cat_features=columnas_category)

In [9]:
model = cb.CatBoostClassifier(**PARAMETROS)

model.fit(entrenamiento_pool)
# make the prediction using the resulting model
preds_temp = model.predict_proba(test_pool)

#Array de probabilidad [0,1] a lista de probabilidades de 1
preds = []
for x in  preds_temp:
    preds.append(x[1])

0:	learn: 0.6423730	total: 132ms	remaining: 794ms
1:	learn: 0.6277057	total: 301ms	remaining: 752ms
2:	learn: 0.6105143	total: 526ms	remaining: 702ms
3:	learn: 0.5951347	total: 643ms	remaining: 482ms
4:	learn: 0.5804733	total: 773ms	remaining: 309ms
5:	learn: 0.5673146	total: 872ms	remaining: 145ms
6:	learn: 0.5529418	total: 984ms	remaining: 0us


In [10]:
if not (PREDICCION_REAL):
    print(log_loss(test_label, preds))

In [11]:
if not (PREDICCION_REAL):
    resultados = pd.DataFrame()
    resultados['Opportunity_ID'] = test['Opportunity_ID']
    resultados['Target'] = pd.Series(preds)
    resultados = resultados.groupby('Opportunity_ID').mean()
    resultados = resultados.reset_index()
    #resultados['Target'] = resultados['Target'].apply(lambda x: int(x >= 0.5))    
    resultados.to_csv("../Archivos/prediccion_arbol_validacion.csv", index=False)
    resultados['Target'].value_counts()

In [12]:
model.get_feature_importance(prettified=True)

,Feature Id,Importances
0,Opportunity_Name,30.772100
1,Bureaucratic_Code,16.538708
2,Planned_Opportunity_Duration,16.198698
3,Product_Type,8.408218
4,Total_Taxable_Amount_Currency,4.270816
...,...,...
77,Total_Amount_by_Product_Family_std,0.000000
78,Buro_Approved_by_Product_Family,0.000000
79,Family_Duration,0.000000
80,Region_Duration,0.000000


In [13]:
if (PREDICCION_REAL):
    resultados = pd.DataFrame()
    resultados['Opportunity_ID'] = test['Opportunity_ID']
    resultados['Target'] = pd.Series(preds)
    resultados = resultados.groupby('Opportunity_ID').mean()
    resultados = resultados.reset_index()
    #resultados['Target'] = resultados['Target'].apply(lambda x: int(x >= 0.5))    
    resultados.to_csv("../Archivos/prediccion_arbol_test.csv", index=False)
    resultados['Target'].value_counts()

In [14]:
preds = model.predict(entrenamiento_pool)
log_loss(objetivo, preds)

3.5442568162376467

In [15]:
if(not MAXIMIZAR_HIPERPARAMETROS): sys.exit()

SystemExit: 

D:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3426: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
from catboost import CatBoostClassifier
from skopt import BayesSearchCV
from skopt.space import Real
from sklearn.model_selection import StratifiedKFold

# Classifier
bayes_cv_tuner = BayesSearchCV(
estimator = CatBoostClassifier(
silent=True,cat_features=columnas_category,task_type="GPU", devices='0:1', bootstrap_type='MVS'
),
search_spaces = {
        "iterations" : (2, 10),
        'learning_rate': Real(low=0.01, high=1, prior='log-uniform'),
        "random_seed" : (1,40000),
        "l2_leaf_reg" : Real(low=1e-9, high=1000, prior='log-uniform'),
        'subsample': Real(low=0.01, high=1, prior='uniform'),
        "random_strength" : Real(low=1e-9, high=1000, prior='log-uniform'),
        'depth': (1, 10),
        #"rsm" : Real(low=0.01, high=1, prior='uniform'),
        "early_stopping_rounds" : (1, 5),
        "border_count" : (1,65535),
        "bagging_temperature" : Real(low=1e-9,high=1000,prior='log-uniform'),
        "scale_pos_weight" : Real(low = 0.01,high = 1.0,prior="uniform")
},
cv = skl.model_selection.TimeSeriesSplit(),
n_jobs = 1,
n_iter = 6,
verbose = 1,
refit = True,
random_state = 72
)

In [ ]:
def status_print(optim_result):
    """Status callback durring bayesian hyperparameter search"""
    # Get all the models tested so far in DataFrame format
    all_models = pd.DataFrame(bayes_cv_tuner.cv_results_)    

    # Get current parameters and the best parameters    
    best_params = pd.Series(bayes_cv_tuner.best_params_)
    print('Model #{}\nBest ROC-AUC: {}\nBest params: {}\n'.format(
        len(all_models),
        np.round(bayes_cv_tuner.best_score_, 4),
        bayes_cv_tuner.best_params_
    ))

In [ ]:
resultCAT = bayes_cv_tuner.fit(entrenamiento, objetivo, callback=status_print)

In [ ]:
print(bayes_cv_tuner.best_score_)
print(bayes_cv_tuner.best_params_)

In [ ]:
preds = bayes_cv_tuner.predict(test.values)

In [ ]:
accuracy_score(test_label, preds)